# Imports

In [ ]:
import json
import numpy as np
import torch
from data.CanopyDataset import CanopyDataset
from model.SEResNet import SEResNet
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load Trained Model

In [12]:
# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SEResNet()
model.load_state_dict(torch.load("../data/model/best_model.pth", map_location=device))
model.to(device)
model.eval()

SEResNet(
  (initial_conv): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (encoder): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      (norm1): GroupNorm(4, 64, eps=1e-05, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      (norm2): GroupNorm(4, 64, eps=1e-05, affine=True)
      (se): SEBlock(
        (se): Sequential(
          (0): AdaptiveAvgPool2d(output_size=1)
          (1): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
          (2): SiLU()
          (3): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1))
          (4): Sigmoid()
        )
      )
      (activation): SiLU()
      (residual_conv): Conv2d(12, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=refl

In [13]:
# Load trained statistics
with open('../data/model/train_stats.dat', 'r') as f:
    stats = json.load(f)
    stats['band_means'] = np.array(stats['band_means'])
    stats['band_stds'] = np.array(stats['band_stds'])

# Evaluate on the test set

In [18]:
def evaluate_model(split, stats=stats):
    dataset = CanopyDataset(split=split, stats_file="../data/model/train_stats.dat")
    loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)

    all_labels, all_preds, all_masks = [], [], []

    with torch.no_grad():
        for images, labels, mask in loader:
            images = images.to(torch.float32).to(device)
            labels = labels.to(torch.float32).to(device)
            mask = mask.to(torch.float32).to(device)

            outputs = model(images)

            all_labels.append(labels.cpu().numpy() * stats['label_std'] + stats['label_mean'])
            all_preds.append(outputs.cpu().numpy() * stats['label_std'] + stats['label_mean'])
            all_masks.append(mask.cpu().numpy())

    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)
    all_masks = np.concatenate(all_masks, axis=0)

    valid_pixels = all_masks > 0
    y_true = all_labels[valid_pixels]
    y_pred = all_preds[valid_pixels]

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"{split.capitalize()} MAE: {mae:.4f}")
    print(f"{split.capitalize()} RMSE: {rmse:.4f}")
    print(f"{split.capitalize()} R² Score: {r2:.4f}")

## Evaluate train split

In [19]:
evaluate_model("train")

Train MAE: 2.3826
Train RMSE: 3.1999
Train R² Score: 0.8403


## Evaluate validation split

In [20]:
evaluate_model("validation")

Validation MAE: 3.3465
Validation RMSE: 4.2889
Validation R² Score: 0.7934


## Evaluate test split

In [21]:

evaluate_model("test")

Test MAE: 3.2891
Test RMSE: 4.2295
Test R² Score: 0.8927
